## Step 1: Import Necessary Libraries

In [ ]:
!pip install llama-index
!pip install llama-index-embeddings-huggingface
!pip install llama-index-packs-raft-dataset

In [2]:
from llama_index.packs.raft_dataset import RAFTDatasetPack
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

For the data preparation process for Q/A generation, the `RAFTDatasetPack` is configured with the following parameters:
- **filepath**: Specifies the path of the file used to generate questions and answers. This file acts as the primary source of content for the dataset.
- **llm**: Defines the Large Language Model (LLM) employed for generating questions and answers. GPT-4 is used by default if no model is specified. Choose the model carefully by considering the costs.
- **embed-model**: The embedding model used to calculate the similarity between a query and its context, essential for selecting relevant context chunks.
- **num_questions_per_chunk**: It determines the number of questions to be created for each data chunk, directly affecting the comprehensiveness of the training dataset.
- **num_distract_docs**: Sets the number of random context chunks used as distractors for each question, challenging the model to identify relevant information.
- **chunk_size**: Llama-index uses SemanticSplitterNodeParser to split the dataset into chunks. So, this parameter is not useful.
- **default_breakpoint_percentile_threshold**: Controls the threshold for combining chunks based on their dissimilarity. A higher value results in fewer, larger chunks, affecting the granularity of the data used for training.

## Step 2: Load OpenAI API Key

In [3]:
import os

os.environ["OPENAI_API_KEY"] = "<YOUR OPENAI API KEY>"

## Step 3: Define LLM and Embedding Models

In [4]:
llm = OpenAI(model="gpt-4")
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## Step 4: Input Dataset

Upload the file that should be used as the 'oracle document' from which the LLM can deduce domain knowlegde. This file will be used to create the RAFT dataset.

*Note: The input data should be pdf, json, txt or api.*

In [5]:
from google.colab import files
uploaded = files.upload()

input_file = input("Enter the name of the uploaded file (including extension): ")

Saving leiden_guidelines.pdf to leiden_guidelines.pdf
Enter the name of the uploaded file (including extension): leiden_guidelines.pdf


The `RAFTDatasetPack` LlamaPack uses the following methods to prepare the data for fine-tuning the LLM:
- **Dividing Sample Data**: Segments the sample data into chunks. Each chunk represents a potential source of information or context for generating questions.
- **Generating Questions**: Creates corresponding questions for every chunk of data. These questions are designed to be answerable using the information within the chunk.
- **Generate answer with Oracle Context**: The 'oracle context' refers to the chunk of data that contains the precise information needed to answer a given question. Uses this context alongside the question to generate the answer using Chain of Thought prompting.
- **Selecting Distractor Contexts**: In addition to the oracle context, a few random chunks of data are chosen as 'distractor contexts'.These simulate noise and irrelevant information, challenging the model to focus on the relevant context.
- **Compiling Training Data**: Compilers include the question, oracle context, distractor contexts, and the generated answer, alongside explicit instructions on how the model should discern and utilize the relevant context to answer questions, into a comprehensive training dataset.
- **Fine-Tuning the Model**: Utilizing this dataset, the model undergoes fine-tuning, learning to accurately distinguish relevant from irrelevant information and to generate precise answers based on the context provided.

In [6]:
# Create RAFT Dataset object
raft_dataset = RAFTDatasetPack(file_path=input_file,
                               llm=llm, embed_model=embed_model,
                               num_questions_per_chunk=1, num_distract_docs=2, chunk_size=1024,
                               default_breakpoint_percentile_threshold=99)

# Beware of the costs invloved from using the OpenAI API.
# It will also take long time based on the file size.
dataset = raft_dataset.run()

# Save the dataset in jsonl format
output_path = './raft_dataset'
dataset.to_json(output_path + ".jsonl")

INFO:llama_index.packs.raft_dataset.base:Number of chunks created: 108
INFO:llama_index.packs.raft_dataset.base:Processing chunk: 0
INFO:llama_index.packs.raft_dataset.base:Processing chunk: 1
INFO:llama_index.packs.raft_dataset.base:Processing chunk: 2
INFO:llama_index.packs.raft_dataset.base:Processing chunk: 3
INFO:llama_index.packs.raft_dataset.base:Processing chunk: 4
INFO:llama_index.packs.raft_dataset.base:Processing chunk: 5
INFO:llama_index.packs.raft_dataset.base:Processing chunk: 6
INFO:llama_index.packs.raft_dataset.base:Processing chunk: 7
INFO:llama_index.packs.raft_dataset.base:Processing chunk: 8
INFO:llama_index.packs.raft_dataset.base:Processing chunk: 9
INFO:llama_index.packs.raft_dataset.base:Processing chunk: 10
INFO:llama_index.packs.raft_dataset.base:Processing chunk: 11
INFO:llama_index.packs.raft_dataset.base:Processing chunk: 12
INFO:llama_index.packs.raft_dataset.base:Processing chunk: 13
INFO:llama_index.packs.raft_dataset.base:Processing chunk: 14
INFO:llam

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

1012268

## Step 5: Loading Dataset

The complete dataset can now be loaded and contains the following keys:
- `id`: Unique identifier for the data point.
- `type`: Type of document.
- `question`: The question generated for this particular chunk of data.
- `context`: The chunk of data from which the model sources the question.
- `oracle_context`: The chunk of data that contains the information needed to answer the question.
- `cot_answer`: The Chain of Thought answer to the generated question.
- `instruction`: The prompt given to the LLM which tells it to answer the question based on the context.

In [7]:
import json

with open('./raft_dataset.jsonl', 'r') as json_file:
    dataset = list(json_file)

# We can access the dataset with the following keys
json.loads(dataset[0]).keys()

# Example: accessing one of the generated questions
json.loads(dataset[0])['question']

'Who supervised the creation of the Leiden Guidelines on the Use of Digitally Derived Evidence in International Criminal Courts and Tribunals?'

Once the dataset is prepared, follow these [instructions](https://github.com/ShishirPatil/gorilla/blob/main/raft/azure-ai-studio-ft/howto.md) to finetune and deploy your own RAFT model. Make sure to use `prompt` as input and `completion` as output when fine tuning a `completion` model and the `messages` column as input when fine tuning a `chat` model.